In [1]:
# Contestant Techniques 
from sklearn.cluster import AgglomerativeClustering
import numpy
from igraph import *
import igraph
import random
import numpy as np
import pandas as pd
from sklearn.metrics.cluster import adjusted_rand_score

path = "/Users/mcdicjh2/Desktop/gecco22/"

for it in range(0,30):

    # Read in graph and associated data
    g_org = Graph.Read_Edgelist(path+"Graph"+str(it)+".dat")
    truth = pd.read_csv(path+"Graph"+str(it)+".truth.dat", sep=',',header=None)
    groundtruth=truth.iloc[:, 1].values.tolist()
    v = pd.read_csv(path+"Graph"+str(it)+".vertices.dat", sep=',',header=None)
    vertices=v.iloc[:, 1].values.tolist()
    edges = g_org.get_edgelist()
    
    proj0 = [i for i,val in enumerate(vertices) if val==0]
    proj1 = [i for i,val in enumerate(vertices) if val==1]
    
    # Generate Bipartite graph
    g3 = Graph.Bipartite(vertices, edges)
    n_var = len(vertices)
    lower = vertices.count(0) 


    c= g3.clusters()  # This should always be 1 one now
    g1, g2 = g3.bipartite_projection(multiplicity=True)

    # Run Multi-Level algorithm (not implemented in igraph package)
    multilevel_result = pd.DataFrame(columns=['k','m','m1','m2','ari'])
    test1 = g1.community_multilevel()
    test2 = g2.community_multilevel()
    
    # Consider perturbation in creating new membership vector
    it1=0
    it2=0
    assignment=[0] * len(vertices)
    for vit in range(0,len(vertices)):
        if vertices[vit] == 0:
            assignment[vit] = test1.membership[it1]
            it1=it1+1
        else:
            assignment[vit] = test2.membership[it2]
            it2=it2+1

    # Cluster assignment from each projection
    #assignment=test1.membership+test2.membership


    k1=max(test1.membership)+1
    k2=max(test2.membership)+1
    d = numpy.zeros(shape=(k1+k2,k1+k2))
    #print(k1)
    #print(k2)
    #print(assignment)

    # Calculate dissimilarity matrix between communities
    for ei in range(0,len(edges)):
        #print(edges[ei],edges[ei][0],edges[ei][1])
        index1=assignment[edges[ei][0]]
        index2=k1+assignment[edges[ei][1]]
        
        if vertices[edges[ei][0]] == 0:
            index1=assignment[edges[ei][0]]
            index2=k1+assignment[edges[ei][1]]
        else:
            index1=k1+assignment[edges[ei][0]]
            index2=assignment[edges[ei][1]]
        
       # print(index1,index2)
        d[index1][index2]=d[index1][index2]+1
        d[index2][index1]=d[index2][index1]+1

    for d1 in range(0,k1+k2):
        for d2 in range(0,k1+k2):
            d[d1][d2] = 1.0/(1.0+d[d1][d2])
        d[d1][d1] = 0


    ctr=0
    for k in range(len(c),k1+k2):
  
        # Run hierarchical clustering on communities
        clustering = AgglomerativeClustering(n_clusters=k,linkage='average',affinity='precomputed').fit(d)
        labels = clustering.labels_


        newlabels = np.empty(n_var)
        newlabels.fill(0)

        for v in range(0,n_var):
            if vertices[v] == 0:
                newlabels[v] = labels[assignment[v]]
            else:
                newlabels[v] = labels[k1+assignment[v]]
        #for v in range(0,lower):
        #    newlabels[v] = labels[assignment[v]]

        #for v in range(lower,n_var):
        #    newlabels[v] = labels[k1+assignment[v]]

        
        proj0_labels=[newlabels[i] for i in proj0]
        proj1_labels=[newlabels[i] for i in proj1]

        m3 = g3.modularity(newlabels)
        m1 = g1.modularity(proj0_labels,weights=g1.es['weight'])
        m2 = g2.modularity(proj1_labels,weights=g2.es['weight'])
        ari = adjusted_rand_score(groundtruth,newlabels)
        multilevel_result.loc[ctr] = k, m3, m1, m2, ari
        ctr=ctr+1


    # Run FastGreedy and Comunity Edge for comparison

    communityedge_result = pd.DataFrame(columns=['k','m','m1','m2','ari'])
    communityedge_res = g3.community_edge_betweenness()

    walktrap_result = pd.DataFrame(columns=['k','m','m1','m2','ari'])
    walktrap_res = g3.community_walktrap()

    fastgreedy_result = pd.DataFrame(columns=['k','m','m1','m2','ari'])
    fastgreedy_res = g3.community_fastgreedy()


   
    for i in range (0,1):
        for k in range(len(c),min(len(c)+15,n_var)):
            
            
            test = fastgreedy_res.as_clustering(k)
            m3 = g3.modularity(test)

            proj0_labels=[test.membership[i] for i in proj0]
            proj1_labels=[test.membership[i] for i in proj1]

            m1 = g1.modularity(proj0_labels,weights=g1.es['weight'])
            m2 = g2.modularity(proj1_labels,weights=g2.es['weight'])
            

            ari = adjusted_rand_score(groundtruth,test.membership)
            fastgreedy_result.loc[i*30+k] = k, m3, m1, m2, ari

            test = communityedge_res.as_clustering(k)
            m3 = g3.modularity(test)
            
            proj0_labels=[test.membership[i] for i in proj0]
            proj1_labels=[test.membership[i] for i in proj1]

            m1 = g1.modularity(proj0_labels,weights=g1.es['weight'])
            m2 = g2.modularity(proj1_labels,weights=g2.es['weight'])


            ari = adjusted_rand_score(groundtruth,test.membership)
            communityedge_result.loc[i*30+k] = k, m3, m1, m2, ari

            test = walktrap_res.as_clustering(k)
            m3 = g3.modularity(test)
            proj0_labels=[test.membership[i] for i in proj0]
            proj1_labels=[test.membership[i] for i in proj1]

            m1 = g1.modularity(proj0_labels,weights=g1.es['weight'])
            m2 = g2.modularity(proj1_labels,weights=g2.es['weight'])

            ari = adjusted_rand_score(groundtruth,test.membership)
            walktrap_result.loc[i*30+k] = k, m3, m1, m2, ari
        
    multilevel_result.to_csv(path+"Graph"+str(it)+".multilevel.csv")
    communityedge_result.to_csv(path+"Graph"+str(it)+".communityedge.csv")
    walktrap_result.to_csv(path+"Graph"+str(it)+".walktrap.csv")
    fastgreedy_result.to_csv(path+"Graph"+str(it)+".fastgreedy.csv")







In [2]:
# Yet another contestant technique
# Use of BRIM package for comparison

import condor
import pandas as pd


for it in range(0,30):


    # Read in graph and associated data
    g_org = Graph.Read_Edgelist(path+"Graph"+str(it)+".dat")
    truth = pd.read_csv(path+"Graph"+str(it)+".truth.dat", sep=',',header=None)
    groundtruth=truth.iloc[:, 1].values.tolist()
    v = pd.read_csv(path+"Graph"+str(it)+".vertices.dat", sep=',',header=None)
    vertices=v.iloc[:, 1].values.tolist()
    edges = g_org.get_edgelist()

    g3 = Graph.Bipartite(vertices, edges)
    n_var = len(vertices)
    lower = vertices.count(0)

    g1, g2 = g3.bipartite_projection(multiplicity=True)
    
    el = g3.get_edgelist()
    
    # Fix edgelist representation for BRIM package (needs 0 vertices as start)
    #print(el)
    #print(vertices)
    for i in range (0,len(el)):
        if vertices[el[i][0]] == 0:
            temp = el[i]
            el[i] = (temp[0], temp[1])
        elif vertices[el[i][1]] == 0:
            temp = el[i]
            el[i] = (temp[1], temp[0])
        else:
            print("Error")

    #print(el)
    net = pd.DataFrame(el)

    
    

    brim_result = pd.DataFrame(columns=['k','ari','m1','m2','m'])

    for i in range (0,1):

        co = condor.condor_object(net)
        co = condor.initial_community(co)
        co = condor.brim(co)

        n_l = vertices.count(0)
        
        #groundtruth1 = groundtruth[0:n_l]
        #groundtruth2 = groundtruth[n_l:n_var]
        

        output1=co["reg_memb"]
        output1=output1["com"].tolist()
        output2 = co["tar_memb"]
        output2=output2["com"].tolist()

        
        
        #ari = adjusted_rand_score(groundtruth1,output2)
        #ari2 = adjusted_rand_score(groundtruth2,output1)

        output3 = np.empty(len(vertices))
        output3.fill(-1)
        index1=0
        index2=0
        for v in range (0,len(vertices)):
            if vertices[v]==0:
                output3[v] = output2[index1]
                index1 = index1+1
            else:
                output3[v] = output1[index2]
                index2 = index2+1
            
        
        ari3 = adjusted_rand_score(groundtruth,output3)
        #print(max(output3)+1,ari,ari2,ari3)

        m = g3.modularity(output3)
        m1 = g1.modularity(output2,weights=g1.es['weight'])#
        m2 = g2.modularity(output1,weights=g2.es['weight'])

        k = (max(output3)+1)
        brim_result.loc[i] = k, ari3, m1, m2, m
        
    brim_result.to_csv(path+"Graph"+str(it)+".brim.csv")


Unweighted network. Weights initialized as 1.
Condor object built in 0.011210918426513672
Initial community structure computed in  0.002141237258911133 . Modularity =  0.5401365000000001
Matrices computed in 0.002660989761352539
0.553517
0.553517
Unweighted network. Weights initialized as 1.
Condor object built in 0.007500886917114258
Initial community structure computed in  0.003259897232055664 . Modularity =  0.5445035
Matrices computed in 0.003702878952026367
0.5626490000000002
0.5626490000000002
Unweighted network. Weights initialized as 1.
Condor object built in 0.005759000778198242
Initial community structure computed in  0.0027430057525634766 . Modularity =  0.5530355000000002
Matrices computed in 0.0025568008422851562
0.570975
0.570975
Unweighted network. Weights initialized as 1.
Condor object built in 0.0057680606842041016
Initial community structure computed in  0.003534078598022461 . Modularity =  0.54765
Matrices computed in 0.002496004104614258
0.568781
0.568781
Unweighte